# Классификация текстов

Сегодня мы будем делать бинарную классификацию текстов, используя kNN или наивный Байесовский классификатор и мешок слов.

### Задача

Мы будем учить классификатор отличать тексты из учебников программирования от библейских текстов. 

### Данные

Скачайте данные по [ссылке](https://drive.google.com/file/d/10vgOvk10yMelNlFAOdGt3YZG0icqp2TI/view?usp=sharing).

In [1]:
import pandas as pd

Загрузим данные в датасет:

In [2]:
# считываем данные из скачанного csv-файла
df = pd.read_csv('bible_vs_coding.csv')

In [3]:
# посмотрим на первые 10
df.head(10)

,labels,sentences
0,testament,И к кому из святых обратишься ты?
1,testament,"И Зелфа, служанка Лиина, [зачала и] родила Иак..."
2,testament,"И когда умрет какой-либо скот, который употреб..."
3,coding,"(Не забывайте, что два сигнала,\nSIGKILL и SI..."
4,testament,"В седьмой день встали рано, при появлении зари..."
5,testament,"И воспылал гнев Господа на Израиля, и водил Он..."
6,testament,"И сошел на него Дух Господень, и веревки, бывш..."
7,testament,2\n\nИ пришел Ангел Господень из Галгала в Бох...
8,coding,Инварианты циклов\nДля всех трех вариантов лин...
9,coding,Поскольку выполняемые операции выполняются от\...


In [4]:
# df.sentences -- колонка с названием "sentences"
# смотрим на третий элемент этой колонки:
print(df.sentences.iloc[2])

И когда умрет какой-либо скот, который употребляется вами в пищу, то прикоснувшийся к трупу его нечист будет до вечера;

и тот, кто будет есть мертвечину его, должен омыть одежды свои и нечист будет до вечера; и тот, кто понесет труп его, должен омыть одежды свои и нечист будет до вечера.


### Векторизация

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
# инициализируем объект векторизатора, он будет запоминать порядок слов и составлять таблички признаков
toy_vectorizer = CountVectorizer()

У объекта веткоризатора, `.fit` -- метод, который смотрит на все слова в текстах и фиксирует их порядок.

In [9]:
toy_vectorizer.fit(['мягкие, мягкие коты', 'коты бегают', 'бегают'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

Теперь векторизатор помнит, какому уникальному слову соответствует какой порядковый номер:

In [11]:
toy_vectorizer.vocabulary_

{'бегают': 0, 'коты': 1, 'мягкие': 2}

А теперь построим саму табличку, в которой посчитано, сколько каждое слово вошло в каждый текст.

Каждая строка в табличке -- текст, каждый столбец -- слово; слова выстроены по алфавиту, и стоят в том порядке, в котором они пронумерованы в словаре `toy_vectorizer.vocabulary_`.

In [13]:
# .transform строит табличку
toy_vectors = toy_vectorizer.transform(['мягкие, мягкие коты', 'коты бегают', 'бегают'])

Таблица строится в sparse формате. Это значит, что:
* сразу вам таблицу не покажут, покажут только сколько в ней строк и столбцов
* она в принципе хранит в себе значения только ненулевых элементов, поэтому занимает мало памяти

In [14]:
toy_vectors

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [15]:
# .todense превращает sparse-матрицу в обучную, на которую можно посмотреть глазами
toy_vectors.todense()

matrix([[0, 1, 2],
        [1, 1, 0],
        [1, 0, 0]])

In [16]:
# обратите внимание, для слова мурлыкают ничего не посчиталось, потому что его не было в словаре
test_vectors = toy_vectorizer.transform(['коты мурлыкают', 'коты мягкие', 'бегают'])
test_vectors.todense()

matrix([[0, 1, 0],
        [0, 1, 1],
        [1, 0, 0]])

#### А теперь применим этот векторизатор к нащим текстам для создания таблицы признаков:

In [13]:
vec = CountVectorizer()

In [14]:
bag_of_words = vec.fit_transform(df.sentences)

In [15]:
bag_of_words

<24902x44976 sparse matrix of type '<class 'numpy.int64'>'
	with 387538 stored elements in Compressed Sparse Row format>

### Обучение

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.labels)

#### Логистическая регрессия
![lr.png](https://static.javatpoint.com/tutorial/machine-learning/images/logistic-regression-in-machine-learning.png)

Формула функции:

$$f(x) = \frac{1}{1 - e^{-x}}$$ Где $$x = w_1 * x_1 + w_2 * x_2 * ...$$

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
lr = LogisticRegression()

In [49]:
clf = lr.fit(X_train, y_train)

/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [50]:
my_text = vec.transform(['Господь, директория каталог сортировка', 'мама мыла рама'])
my_text

<2x44976 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [51]:
clf.predict(my_text)

array(['testament', 'coding'], dtype=object)

In [52]:
clf.predict_proba(my_text)

array([[0.33679326, 0.66320674],
       [0.74948559, 0.25051441]])

### Оценка

In [53]:
from sklearn.metrics import classification_report

In [54]:
predicted_labels = clf.predict(X_test)

In [55]:
predicted_labels

array(['testament', 'coding', 'testament', ..., 'coding', 'coding',
       'testament'], dtype=object)

In [56]:
print(classification_report(y_test, predicted_labels))

              precision    recall  f1-score   support

      coding       0.98      0.99      0.98      2844
   testament       0.99      0.98      0.99      3382

    accuracy                           0.99      6226
   macro avg       0.98      0.99      0.99      6226
weighted avg       0.99      0.99      0.99      6226

